In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF

In [2]:
survey=pd.read_csv('/home/ayca/qmbu450/hw4/immSurvey.csv')
survey.head()

,Unnamed: 0,ids,X.1,textToSend,Means,stanMeans,X,MetaID,treatment,pid_rep,...,RunAll,mergeIDs,stanMeansNewSys,stanSDsNewSys,coderMeans,newDocIDS,stanMeansNewSys2,stanSDsNewSys2,stanMeansNewSysPooled,stanSDsNewSysPooled
0,1,6606,1,problems caused by the influx of illegal immig...,0.750000,2.409768,1,0,1,1.00000,...,0.750291,240,1.919906,0.549770,1.5,10335,1.106977,0.320538,1.465566,0.288474
1,2,6607,2,"if you mean illegal immigration, i'm afraid of...",0.875000,3.710615,2,0,1,1.00000,...,1.011095,75,-0.138688,0.302764,1.0,10336,-0.223681,0.300575,-0.166691,0.219645
2,3,6608,3,that they should enter the same way my grandpa...,0.416667,-1.437706,3,0,0,0.33300,...,-0.219371,271,-0.334633,0.291336,0.0,10337,0.825119,0.317955,0.224096,0.220390
3,4,6609,4,legally entering the usa meeting the requireme...,0.458333,0.655503,4,0,0,0.50000,...,0.329845,181,0.386023,0.294324,1.5,10338,0.679629,0.305606,0.629995,0.221096
4,5,6610,5,terror bombings killing us robbing america,0.875000,5.337525,5,0,1,0.66667,...,1.955240,261,0.102707,0.274519,2.0,10339,1.206389,0.370577,0.574416,0.216228


In [3]:
alphas = survey.stanMeansNewSysPooled
sample = survey.textToSend

In [4]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sample)
pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names()).head()

,11,12,125,18,1b,600,95,able,abolition,aboration,...,wreckless,wrong,wrongly,xenophobia,years,yet,you,young,your,yrars
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sample)
pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names()).head()

,11,12,125,18,1b,600,95,able,abolition,aboration,...,wreckless,wrong,wrongly,xenophobia,years,yet,you,young,your,yrars
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.253915,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [6]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, alphas, random_state=1)

In [7]:
rbf = ConstantKernel(1.0) * RBF(length_scale=1.0)
gpr = GaussianProcessRegressor(kernel=rbf, alpha=1e-8)

In [8]:
gpr.fit(Xtrain.toarray(), ytrain)

GaussianProcessRegressor(alpha=1e-08, kernel=1**2 * RBF(length_scale=1))

In [9]:
mu_s, cov_s = gpr.predict(Xtest.toarray(), return_cov=True)

In [10]:
np.corrcoef(ytest, mu_s)

array([[1.        , 0.68328523],
       [0.68328523, 1.        ]])

In [11]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b')
X = bigram_vectorizer.fit_transform(sample)
pd.DataFrame(X.toarray(), columns=bigram_vectorizer.get_feature_names()).head()

,1,1 3,1 became,1 boarders,1 difficult,1 immigrants,1 is,1 language,1 not,11,...,you think,you want,young,young from,your,your poor,your tired,your weak,yrars,yrars ago
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, alphas, random_state=1)

In [13]:
rbf = ConstantKernel(1.0) * RBF(length_scale=1.0)
gpr = GaussianProcessRegressor(kernel=rbf, alpha=1e-8)

In [14]:
gpr.fit(Xtrain.toarray(), ytrain)

GaussianProcessRegressor(alpha=1e-08, kernel=1**2 * RBF(length_scale=1))

In [15]:
mu_s, cov_s = gpr.predict(Xtest.toarray(), return_cov=True)

In [16]:
np.corrcoef(ytest, mu_s)

array([[1.        , 0.60884464],
       [0.60884464, 1.        ]])

Correlation was 0.68328523. After bigrams included, correlation became 0.60884464.